In [1]:
import pickle
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from modeling_gemma import KVCache
from processing_paligemma import PaliGemmaProcessor, LabelProcessor
from utils import move_inputs_to_device
from prepare_data import ImageInstructionOutputDataset, update_tokenizer, update_embeddings, prepare_dataset
from utils import *
from datetime import datetime

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = 'cpu'
print("Device in use: ", device)
model_path = 'paligemma'

Device in use:  cuda


In [2]:
print(f"Loading model")
tokenizer_modified = True
rank = 16  # LoRA rank
# model, tokenizer = load_hf_model(model_path, device, 1)
bit=4
model, tokenizer = load_hf_model(model_path, freeze_vision=0,bit=bit)
try_round = 'color_3'
# model = apply_lora(model, device, rank=rank)
# model = model.half()
model = model.to(device).eval()
vocab_size = len(tokenizer)
cp_epoch = 39

Loading model


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You shouldn't move a model that is dispatched using accelerate hooks.


trainable params: 18,432,000 || all params: 2,527,094,784 || trainable%: 0.7294


In [3]:
update_embeddings(model, model.vocab_size, vocab_size, device)
num_image_tokens = model.config.vision_config.num_image_tokens
image_size = model.config.vision_config.image_size
processor = PaliGemmaProcessor(tokenizer, num_image_tokens, (image_size, image_size))
label_processor = LabelProcessor(tokenizer)
stop_token = processor.tokenizer.eos_token_id

Update embeddings


In [4]:
batch_size = 6
training_set, validation_set = prepare_dataset(bs=batch_size)
colors_dict = list(np.unique(training_set.baseColour))
colors_dict = {color: i for i, color in enumerate(colors_dict)}

cate_dict = list(np.unique(training_set.subCategory))
cate_dict = {cate: i for i, cate in enumerate(cate_dict)}

Data augmenting...
35551/35552
Viewing the set into batch_size=6
Data augmenting...
4443/4444
Viewing the set into batch_size=6


In [29]:
for n,m in model.named_modules():
    if hasattr(m, 'weight'):
#         print(m.weight.dtype)
        m.weight.

torch.float16
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.float16
torch.int8
torch.int8
torch.int8

In [5]:
checkpoint = f'my_model/5/model_rank16_epoch_38_.pth'
model.load_state_dict(torch.load(checkpoint))

<All keys matched successfully>

In [ ]:
'''
多loss训练
'''
from sklearn.metrics import accuracy_score, f1_score

try_round = '5'


# for name, m in model.named_parameters():  # 只更新后两层
#     if ('vision_tower.vision_model.encoder.layers.26' in name or 'vision_tower.vision_model.encoder.layers.25' in name or 'post_layernorm' in name) and 'original' not in name:
#         m.requires_grad = True
#     else:
#         m.requires_grad = False

class GlobalClassifyingHead(nn.Module):
    def __init__(self,
                 in_dim=1152,  # 输入特征维度
                 compress_dim=512,  # 特征压缩维度
                 num_cates=17):  # 类别数
        super().__init__()
        # 特征压缩与全局融合
        self.feature_fusion = nn.Sequential(
            nn.Linear(in_dim, compress_dim),
            nn.GELU()
        )
        # 注意力聚合模块（自动聚焦重要区域）
        self.attention_pool = nn.Sequential(
            nn.Linear(compress_dim, 1),  # 为每个patch生成注意力权重
            nn.Softmax(dim=1)
        )
        # 颜色分类器
        self.classifier = nn.Sequential(
            nn.Linear(compress_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_cates))

    def forward(self, x):
        # 特征压缩 [8, 256, 512]
        x = self.feature_fusion(x)
        # 注意力权重计算 [8, 256, 1]
        attn_weights = self.attention_pool(x)
        # 加权聚合 [8, 512]
        global_feature = torch.sum(x * attn_weights, dim=1)
        # 颜色分类 [8, num_colors]
        return torch.softmax(self.classifier(global_feature),-1)


# color_classifier = GlobalClassifyingHead(2048).half().to(device)
# cate_classifier = GlobalClassifyingHead(2048, num_cates=45).half().to(device)
# color_weight = 0.2
# cate_weight = 0.2
# loss_weight = 1 - color_weight - cate_weight
# if cp_epoch > 0:
#     checkpoint_c = f'my_model/' + try_round + f'/color_classifier_epoch_{cp_epoch-1}_.pth'
#     color_classifier.load_state_dict(torch.load(checkpoint_c))
#     checkpoint_c = f'my_model/' + try_round + f'/cate_classifier_epoch_{cp_epoch-1}_.pth'
#     cate_classifier.load_state_dict(torch.load(checkpoint_c))
    
lr = 5e-4
# optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
loss_fn_output = torch.nn.CrossEntropyLoss().to(device)
# loss_fn_color = torch.nn.CrossEntropyLoss().to(device)
# loss_fn_cate = torch.nn.CrossEntropyLoss().to(device)
# loss_fn = torch.nn.MSELoss().to(device)

optimizer = torch.optim.SGD([
    {'params': filter(lambda p: p.requires_grad, model.parameters()), 'lr': lr},  # 冻结前层
#     {'params': color_classifier.parameters(), 'lr': lr},
#     {'params': cate_classifier.parameters(), 'lr': lr}
])

# optimizer = torch.optim.SGD(model.parameters(),lr=lr)


# for m in color_classifier.modules():
#     if isinstance(m, (nn.Conv2d)):
#         nn.init.normal_(m.weight,mean=0,std=0.005)
#     if isinstance(m, (nn.Linear)):
#         nn.init.normal_(m.weight,mean=0,std=0.005)
#         nn.init.normal_(m.bias,mean=0,std=0.005)

epochs = 30
max_tokens_to_generate = 100
print('Start training on classifying colors:')
print(f'Start at epoch {cp_epoch}')
torch.cuda.empty_cache()
# torch.cuda.memory._record_memory_history(max_entries=100000)
for epoch in range(epochs):
    epoch += cp_epoch
    correct_num_train = 0
    total_num_train = 0
    loss_list = []
    cate_list_pred, cate_list_true = [], []
    color_list_pred, color_list_true = [], []
    train_assessments = [] if epoch == 0 else pickle.load(open(f'track/'+try_round+'/train_assessments.pkl', 'rb'))
    print('\nShuffle training set')
    training_set.shuffle(batch_size)  # shuffle training set
    for i, (image_paths, prompts, label_words, images, colors, _, cates) in enumerate(training_set):
#         if i == 10:
#             break
        images = [images[:, :, :, i] for i in range(images.shape[3])]
        model_inputs = processor(prompts, images)
        model_inputs = move_inputs_to_device(model_inputs, device)
        input_ids = model_inputs['input_ids'].to(device)
        attention_mask = model_inputs['attention_mask'].to(device)
        pixel_values = model_inputs['pixel_values'].half().to(device)
        kv_cache = KVCache()
        labels_ids, labels_attention_mask, total_num = get_ids(tokenizer, label_words)
        total_num_train += total_num
        labels_ids = labels_ids.to(device)
        labels_attention_mask = labels_attention_mask.to(device)
        generated_words = ''
        loss_list_sample = []
        correct_num = 0
        for j in range(labels_ids.shape[1]):
            outputs, image_embeddings, image_features, hidden_feature = model.multi_forward(input_ids=input_ids,
                                                                            pixel_values=pixel_values,
                                                                            attention_mask=attention_mask,
                                                                            kv_cache=kv_cache)
            kv_cache = outputs["kv_cache"]
            word_ids = labels_ids[:, j].to(device)
            word_vec = torch.zeros_like(outputs["logits"][:, -1, :], dtype=torch.float16)
            word_vec[list(range(batch_size)), word_ids[:, 0]] = 1.0
            input_ids = word_ids.detach()  # 标签的下一个（一个batch）的字符作为下一次输入
            attention_mask = torch.cat([attention_mask, (word_ids != 0).int()], dim=-1).detach().to(
                device)  # 更新attention_mask，若batch中的某一条样本已经输入完，则mask对应的位置为0，否则为1

#             if j == 0:
#                 color_outputs = color_classifier(hidden_feature)
#                 color_ids = list(color_outputs.argmax(dim=1).detach().cpu().numpy())
#                 color_list_pred += color_ids
#                 color_labels = [colors_dict[c] for c in colors]
#                 color_list_true += color_labels
#                 colors_vec = torch.eye(17)[color_labels].to(device)

#                 cate_outputs = cate_classifier(hidden_feature)
#                 cate_ids = list(cate_outputs.argmax(dim=1).detach().cpu().numpy())
#                 cate_list_pred += cate_ids
#                 cate_labels = [cate_dict[c] for c in cates]
#                 cate_list_true += cate_labels
#                 cate_vec = torch.eye(45)[cate_labels].to(device)

#                 color_loss = loss_fn_color(color_outputs, colors_vec)
#                 cate_loss = loss_fn_cate(cate_outputs, cate_vec)
#                 optimizer.zero_grad()
#                 if torch.isnan(color_loss) or torch.isnan(cate_loss):
#                     print('nan')
#             else:
#                 optimizer.zero_grad()
            optimizer.zero_grad()
            loss = loss_fn_output(outputs["logits"][:, -1, :], word_ids[:, 0])
#             if j == 0:
#                 final_loss = loss * loss_weight + color_loss * color_weight + cate_loss * cate_weight
#                 final_loss.backward()
#             else:
#                 loss.backward()
            loss.backward()
            if torch.isnan(loss):
                print('nan')
            else:   
                optimizer.step()
            kv_cache.clear_trace()
            loss = round(float(loss.detach().cpu()), 4)
            pred_ids = torch.softmax(outputs["logits"][:, -1, :], dim=-1, dtype=torch.float16).argmax(1)
            ei = word_ids[:, 0] != 0
            loss_list_sample.append(loss)
#             printing = f'Epoch{epoch}\tIter{i}\t{j}\tPredict Loss:{loss}\t Color Loss:{color_loss}\t Cate Loss:{cate_loss}'
            printing = f'Epoch{epoch}\tIter{i}\t{j}\tPredict Loss:{loss}'
            print(printing, end='\r')
#             break
            correct_num_train += (word_ids[:, 0][ei] == pred_ids[ei]).sum().item()
        loss_list.append(sum(loss_list_sample) / len(loss_list_sample))
        torch.cuda.empty_cache()
#         break
#     train_assessments.append([float(correct_num_train / total_num_train),
#                       accuracy_score(color_list_true, color_list_pred),
#                       f1_score(color_list_true, color_list_pred, average='macro'),
#                       accuracy_score(cate_list_true, cate_list_pred),
#                       f1_score(cate_list_true, cate_list_pred, average='macro')])

    train_assessments.append(float(correct_num_train / total_num_train))
    print('Saving loss, training assessments and the model...')
    torch.save(model.state_dict(),
               f'my_model/{try_round}/model_rank{rank}_epoch_{epoch}_.pth')
#     torch.save(color_classifier.state_dict(),
#                f'my_model/{try_round}/color_classifier_epoch_{epoch}_.pth')
#     torch.save(cate_classifier.state_dict(),
#                f'my_model/{try_round}/cate_classifier_epoch_{epoch}_.pth')
    with open(f'track/{try_round}/loss_rank{rank}_epoch_{epoch}_.pkl', 'wb') as f:
        pickle.dump(loss_list, f)
    with open(f'track/{try_round}/train_assessments.pkl', 'wb') as f:
        pickle.dump(train_assessments, f)

    validation_acc = [] if epoch == 0 else pickle.load(open(f'track/'+try_round+'/validation_acc.pkl', 'rb'))
    with torch.no_grad():
        print(f'Validating after epoch {epoch}')
        correct_num_valid = 0
        total_num_valid = 0
        validation_set.shuffle(4)
        for i, (image_paths, prompts, label_words, images, _, _, _) in enumerate(validation_set):
            print(f'\r{i}/{len(validation_set)}', end='')
            images = [images[:, :, :, i] for i in range(images.shape[3])]
            model_inputs = processor(prompts, images)
            model_inputs = move_inputs_to_device(model_inputs, device)
            input_ids = model_inputs['input_ids'].to(device)
            attention_mask = model_inputs['attention_mask'].to(device)
            pixel_values = model_inputs['pixel_values'].half().to(device)
            kv_cache = KVCache()
            labels_ids, labels_attention_mask, total_num = get_ids(tokenizer, label_words)
            total_num_valid += total_num
            labels_ids = labels_ids.to(device)
            labels_ids[labels_ids==0] = -1
            labels_attention_mask = labels_attention_mask.to(device)
            for j in range(max_tokens_to_generate):
                if j >= labels_ids.shape[1]:
                    break
                outputs, _, _ = model(input_ids=input_ids,
                                        pixel_values=pixel_values,
                                        attention_mask=attention_mask,
                                        kv_cache=kv_cache)
                kv_cache = outputs["kv_cache"]
                pred_ids = outputs["logits"][:, -1, :].argmax(1)
                attention_mask = torch.cat([attention_mask, (labels_ids[:, j, :] != -1).int()], dim=-1).detach().to(device)
                correct_num_valid += (pred_ids == labels_ids[:, j, 0]).sum()
                input_ids = torch.unsqueeze(pred_ids, 1)
    val_acc = float((correct_num_valid/total_num_valid).cpu().numpy())
    validation_acc.append(val_acc)
    print(f'\n\tAccuracy on validation set is {round(val_acc, 4)}')
    print('\tSaving accuracy on validation set\n')
    with open(f'track/{try_round}/validation_acc.pkl', 'wb') as f:
        pickle.dump(validation_acc, f)
    training_set, validation_set = prepare_dataset(bs=batch_size)

# torch.cuda.memory._dump_snapshot(f"track/rank_{rank}_bit_{bit}_GPUMemoryUsage.pickle")
# torch.cuda.memory._record_memory_history(enabled=None)

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Start training on classifying colors:
Start at epoch 39

Shuffle training set
Viewing the set into batch_size=6


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
bit

32

In [9]:
torch.cuda.memory._dump_snapshot("test.pickle")
torch.cuda.memory._record_memory_history(enabled=None)

In [32]:
model.language_model.base_model.model.model.embed_tokens.weight

Parameter containing:
tensor([[ 0.5352, -0.0342,  0.0956,  ...,  0.0394,  0.2649,  0.0719],
        [ 0.1616, -0.1614, -0.1450,  ..., -0.0263,  0.0057, -0.0356],
        [ 0.1212,  0.0206, -0.0285,  ..., -0.0060,  0.0020, -0.0056],
        ...,
        [ 0.3709, -0.3123,  0.1329,  ...,  0.4703, -0.0135, -0.0267],
        [-0.3849,  0.2765, -0.4246,  ...,  0.3767,  0.2968,  0.1435],
        [-0.0473,  0.4165, -0.3479,  ...,  0.0354,  0.1006, -0.0609]],
       device='cuda:0', requires_grad=True)

In [12]:
model.language_model.base_model.model.model.embed_tokens.weight.grad

tensor([[-4.1900e-25, -1.4440e-24, -2.6146e-25,  ...,  8.0674e-25,
          6.8459e-26, -2.4822e-25],
        [ 2.3831e-01,  4.2277e-01,  1.1893e-01,  ...,  6.3680e-02,
          3.8681e-02, -2.3741e-02],
        [-2.5610e-41, -9.7076e-41, -6.1769e-42,  ...,  6.5886e-41,
          8.8114e-42, -2.0620e-41],
        ...,
        [-2.3259e-34, -6.8676e-34, -5.2102e-34,  ...,  9.0979e-35,
         -8.0793e-35,  6.3096e-35],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')

In [7]:
outputs["logits"][:, -1, :]

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [8]:
input_ids

tensor([[257152, 257152, 257152,  ...,      0,      0,      0],
        [257152, 257152, 257152,  ...,      0,      0,      0],
        [257152, 257152, 257152,  ...,      0,      0,      0],
        [257152, 257152, 257152,  ...,      0,      0,      0]],
       device='cuda:0')

In [ ]:
# 262875
# 262888
# 257153
# 262536
# 262852
# 262301

In [19]:
(262301-257153)*6

30888

In [4]:
'''更新tokenizer'''
training_set, validation_set = prepare_dataset(validation=1)
training_set.shuffle(1)
validation_set.shuffle(1)
tokenizer = AutoTokenizer.from_pretrained(model_path)
update_tokenizer(tokenizer, validation_set)
update_tokenizer(tokenizer, training_set)
# update_tokenizer(tokenizer, testing_set)

Data augmenting...
35551/35552
Viewing the set into batch_size=4
Data augmenting...
4443/4444
Viewing the set into batch_size=4
Viewing the set into batch_size=1
Viewing the set into batch_size=1
Update tokenizer
4443
Update tokenizer
35551


In [6]:
tokenizer.save_pretrained(model_path)

('paligemma/tokenizer_config.json',
 'paligemma/special_tokens_map.json',
 'paligemma/tokenizer.json')

In [ ]:
batch_size = 4
lr = 1e-4
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, eps=1e-3)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fn = torch.nn.CrossEntropyLoss().to(device)
epochs = 50
max_tokens_to_generate = 100

In [ ]:
'''
整体LoRA微调
'''
try_round = 3
print('Start training:')
print(f'Start at epoch {cp_epoch}')
for epoch in range(epochs):
    epoch += cp_epoch
    training_set, validation_set = prepare_dataset(validation=1)
    correct_num_train = 0
    total_num_train = 0
    loss_list = []
    train_acc = [] if epoch == 0 else pickle.load(open(f'track/{try_round}/train_acc.pkl', 'rb'))
    print('\nShuffle training set')
    training_set.shuffle(batch_size)  # shuffle training set
    for i, (image_paths, prompts, label_words, images, _, _, _) in enumerate(training_set):
        images = [images[:, :, :, i] for i in range(images.shape[3])]
        model_inputs = processor(prompts, images)
        model_inputs = move_inputs_to_device(model_inputs, device)
        input_ids = model_inputs['input_ids'].to(device)
        attention_mask = model_inputs['attention_mask'].to(device)
        pixel_values = model_inputs['pixel_values'].half().to(device)
        kv_cache = KVCache()
        labels_ids, labels_attention_mask, total_num = get_ids(tokenizer, label_words)
        labels_ids = labels_ids.to(device)
        labels_attention_mask = labels_attention_mask.to(device)
        generated_words = ''
        loss_list_sample = []
        correct_num = 0
        for j in range(labels_ids.shape[1]):
            outputs, _, _ = model(
                input_ids=input_ids,
                pixel_values=pixel_values,
                attention_mask=attention_mask,
                kv_cache=kv_cache, )
            kv_cache = outputs["kv_cache"]
            word_ids = labels_ids[:, j].to(device)
            word_vec = torch.zeros_like(outputs["logits"][:, -1, :], dtype=torch.float16)
            word_vec[list(range(batch_size)), word_ids[:, 0]] = 1.0
            input_ids = word_ids.detach()  # 标签的下一个（一个batch）的字符作为下一次输入
            attention_mask = torch.cat([attention_mask, (word_ids != 0).int()], dim=-1).detach().to(
                device)  # 更新attention_mask，若batch中的某一条样本已经输入完，则mask对应的位置为0，否则为1

            loss = loss_fn(outputs["logits"][:, -1, :], word_ids[:, 0])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            kv_cache.clear_trace()
            loss = loss.item()
            pred_ids = torch.softmax(outputs["logits"][:, -1, :], dim=-1, dtype=torch.float16).argmax(1)
            ei = word_ids[:, 0] != 0
            correct_num += (word_ids[:, 0][ei] == pred_ids[ei]).sum().item()
            loss_list_sample.append(loss)
            print(f'epoch{epoch}\t{i}\tloss: {round(loss, 4)}', end='\r')
        #         print(f'\n\tacc:{round(correct_num / total_num, 4)}\ttokens to predict:{total_num}\tcorrect:{correct_num}')
        total_num_train += total_num
        correct_num_train += correct_num
        loss_list.append(sum(loss_list_sample) / len(loss_list_sample))
        torch.cuda.empty_cache()
    train_acc.append(correct_num_train / total_num_train)
    print('Saving loss, training accuracy and the model...')
    torch.save(model.state_dict(),
               f'my_model/{try_round}/model_{str(datetime.now().date())}_rank{rank}_epoch_{epoch}_.pth')
    with open(f'track/{try_round}/loss_{str(datetime.now().date())}_rank{rank}_epoch_{epoch}_.pkl', 'wb') as f:
        pickle.dump(loss_list, f)
    with open(f'track/{try_round}/train_acc.pkl', 'wb') as f:
        pickle.dump(train_acc, f)

    validation_acc = [] if epoch == 0 else pickle.load(open(f'track/{try_round}/validation_acc.pkl', 'rb'))
    with torch.no_grad():
        print(f'Validating after epoch {epoch}')
        correct_num_valid = 0
        total_num_valid = 0
        validation_set.shuffle(4)
        for i, (image_paths, prompts, label_words, images, _, _, _) in enumerate(validation_set):
            print(f'\r{i}/{len(validation_set)}', end='')
            images = [images[:, :, :, i] for i in range(images.shape[3])]
            model_inputs = processor(prompts, images)
            model_inputs = move_inputs_to_device(model_inputs, device)
            input_ids = model_inputs['input_ids'].to(device)
            attention_mask = model_inputs['attention_mask'].to(device)
            pixel_values = model_inputs['pixel_values'].half().to(device)
            kv_cache = KVCache()
            labels_ids, labels_attention_mask, total_num = get_ids(tokenizer, label_words)
            total_num_valid += total_num
            labels_ids = labels_ids.to(device)
            #         labels_ids[labels_ids==0] -= 1
            labels_attention_mask = labels_attention_mask.to(device)
            for j in range(max_tokens_to_generate):
                if j >= labels_ids.shape[1]:
                    break
                #             total_num_valid += 4 - (labels_ids[:,j,0] == -1).sum()
                outputs, _, _ = model(
                    input_ids=input_ids,
                    pixel_values=pixel_values,
                    attention_mask=attention_mask,
                    kv_cache=kv_cache, )
                kv_cache = outputs["kv_cache"]
                pred_ids = outputs["logits"][:, -1, :].argmax(1)
                #             attention_mask = torch.cat([attention_mask, torch.tensor([[1]]).to(device)], dim=-1).to(device)
                attention_mask = torch.cat([attention_mask, (labels_ids[:, j, :] != 0).int()], dim=-1).detach().to(
                    device)
                correct_num_valid += (pred_ids == labels_ids[:, j, 0]).sum()
                input_ids = torch.unsqueeze(pred_ids, 1)
    #             if pred_ids == stop_token:
    #                 break
    acc_valid = float(correct_num_valid / total_num_valid)
    print(f'\nAccuracy on validation set is {round(acc_valid, 4)}\n')
    validation_acc.append(acc_valid)
    print('Saving accuracy on validation set\n')
    with open(f'track/{try_round}/validation_acc.pkl', 'wb') as f:
        pickle.dump(validation_acc, f)

In [ ]:
batch_size = 64
training_set, validation_set = prepare_dataset(bs=batch_size)
colors_dict = list(np.unique(training_set.baseColour))
colors_dict = {color: i for i, color in enumerate(colors_dict)}

cate_dict = list(np.unique(training_set.subCategory))
cate_dict = {cate: i for i, cate in enumerate(cate_dict)}

In [ ]:
for name, m in model.named_parameters():
    print(name)

In [ ]:
'''
加强颜色感知
'''
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score


class Reshape(nn.Module):
    def __init__(self, shape, exchange_dims):
        super(Reshape, self).__init__()
        self.shape = shape
        self.exchange_dims = exchange_dims

    def forward(self, x):
        return x.view(self.shape).swapdims(self.exchange_dims[0], self.exchange_dims[1])


# checkpoint = f'my_model/2/model_2025-02-27_rank16_epoch_25_.pth'
# model.load_state_dict(torch.load(checkpoint))

for name, m in model.named_parameters():  # 只更新后两层
    if (
            'vision_tower.vision_model.encoder.layers.26' in name or 'vision_tower.vision_model.encoder.layers.25' in name or 'post_layernorm' in name) and 'original' not in name:
        m.requires_grad = True
    else:
        m.requires_grad = False

lr = 1e-3
# optimizer = torch.optim.Adam(, lr=lr, eps=1e-3)


# optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
loss_fn = torch.nn.CrossEntropyLoss().to(device)
# loss_fn = torch.nn.MSELoss().to(device)

epochs = 30
max_tokens_to_generate = 100


class GlobalColorHead(nn.Module):
    def __init__(self,
                 in_dim=1152,  # 输入特征维度
                 compress_dim=512,  # 特征压缩维度
                 num_colors=17):  # 颜色类别数
        super().__init__()
        # 特征压缩与全局融合
        self.feature_fusion = nn.Sequential(
            #             nn.LayerNorm(in_dim),
            nn.Linear(in_dim, compress_dim),  # 降维减少计算量
            nn.GELU(),
            nn.Dropout(0.2)
        )
        # 注意力聚合模块（自动聚焦重要区域）
        self.attention_pool = nn.Sequential(
            nn.Linear(compress_dim, 1),  # 为每个patch生成注意力权重
            nn.Softmax(dim=1)
        )

        # 颜色分类器
        self.classifier = nn.Sequential(
            nn.Linear(compress_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_colors))

    def forward(self, x):
        #         B, H, W, C = x.shape
        #         x = x.view(B, H*W, C)  # [8, 256, 1152]
        # 特征压缩 [8, 256, 512]
        x = self.feature_fusion(x)
        # 注意力权重计算 [8, 256, 1]
        attn_weights = self.attention_pool(x)
        # 加权聚合 [8, 512]
        global_feature = torch.sum(x * attn_weights, dim=1)
        # 颜色分类 [8, num_colors]
        return self.classifier(global_feature)


color_classifier = GlobalColorHead().half().to(device)
if cp_epoch > 0:
    checkpoint_c = f'my_model/' + try_round + '/color_classifier.pth'
    model.load_state_dict(torch.load(checkpoint_c))

optimizer = torch.optim.SGD([
    {'params': filter(lambda p: p.requires_grad, model.parameters()), 'lr': 1e-2},  # 冻结前层
    {'params': color_classifier.parameters(), 'lr': 1e-2}
])

# for m in color_classifier.modules():
#     if isinstance(m, (nn.Conv2d)):
#         nn.init.normal_(m.weight,mean=0,std=0.005)
#     if isinstance(m, (nn.Linear)):
#         nn.init.normal_(m.weight,mean=0,std=0.005)
#         nn.init.normal_(m.bias,mean=0,std=0.005)


print('Start training on classifying colors:')
print(f'Start at epoch {cp_epoch}')
for epoch in range(epochs):
    epoch += cp_epoch
    correct_num_train = 0
    total_num_train = 0
    pred_list_train = []
    true_list_train = []
    train_acc = [] if epoch == 0 else pickle.load(open(f'track/' + str(try_round) + '/train_acc.pkl', 'rb'))
    train_f1 = [] if epoch == 0 else pickle.load(open(f'track/' + str(try_round) + '/train_f1.pkl', 'rb'))
    training_set.shuffle(batch_size)
    for i, (image_paths, prompts, label_words, images, colors, _, _) in enumerate(training_set):
        images = [images[:, :, :, i] for i in range(images.shape[3])]
        model_inputs = processor(prompts, images)
        model_inputs = move_inputs_to_device(model_inputs, device)
        input_ids = model_inputs['input_ids'].to(device)
        pixel_values = model_inputs['pixel_values'].half().to(device)
        image_features = model.distract_image_features(
            input_ids=input_ids,
            pixel_values=pixel_values)
        outputs = color_classifier(image_features)
        outputs_ids = outputs.argmax(dim=1).detach().cpu().numpy()
        colors_ids = [colors_dict[c] for c in colors]
        colors_vec = torch.eye(17)[colors_ids].to(device)
        loss = loss_fn(outputs.to(torch.float32), colors_vec)
        optimizer.zero_grad()
        #         t = torch.sum(image_features)
        loss.backward()
        optimizer.step()
        loss = float(loss.detach().cpu())
        print(f'epoch{epoch}\t{i}\tloss: {round(loss, 4)}', end='\r')
        for c in colors_ids:
            true_list_train.append(c)
        for c in outputs_ids:
            pred_list_train.append(c)
        torch.cuda.empty_cache()
    acc = accuracy_score(true_list_train, pred_list_train)
    f1 = f1_score(true_list_train, pred_list_train, average='macro')
    train_acc.append(acc)
    train_f1.append(f1)
    print(f'\nAccuracy on training set: {round(acc, 4)}\nF1 score on training set: {round(f1, 4)}')
    print('Saving loss, training accuracy and the model...\n')
    torch.save(model.state_dict(),
               f'my_model/' + str(try_round) + f'/model_{str(datetime.now().date())}_rank{rank}_epoch_{epoch}_.pth')
    torch.save(color_classifier.state_dict(), f'checkpoint_c')
    with open(f'track/' + str(try_round) + '/train_acc.pkl', 'wb') as f:
        pickle.dump(train_acc, f)
    with open(f'track/' + str(try_round) + '/train_f1.pkl', 'wb') as f:
        pickle.dump(train_f1, f)

    validation_acc = [] if epoch == 0 else pickle.load(open(f'track/' + str(try_round) + '/validation_acc.pkl', 'rb'))
    validation_f1 = [] if epoch == 0 else pickle.load(open(f'track/' + str(try_round) + '/validation_f1.pkl', 'rb'))
    validation_set.shuffle(batch_size)
    with torch.no_grad():
        print(f'Validating after epoch {epoch}')
        true_list_val = []
        pred_list_val = []
        for i, (image_paths, prompts, label_words, images, colors, _, _) in enumerate(validation_set):
            print(f'\r{i}/{len(validation_set)}', end='')
            images = [images[:, :, :, i] for i in range(images.shape[3])]
            model_inputs = processor(prompts, images)
            model_inputs = move_inputs_to_device(model_inputs, device)
            input_ids = model_inputs['input_ids'].to(device)
            pixel_values = model_inputs['pixel_values'].half().to(device)
            image_features = model.distract_image_features(input_ids=input_ids,
                                                           pixel_values=pixel_values)
            outputs = color_classifier(image_features)
            outputs_ids = outputs.argmax(dim=1).detach().cpu().numpy()
            colors_ids = [colors_dict[c] for c in colors]
            for c in colors_ids:
                true_list_val.append(c)
            for c in outputs_ids:
                pred_list_val.append(c)
    acc = accuracy_score(true_list_val, pred_list_val)
    f1 = f1_score(true_list_val, pred_list_val, average='macro')
    validation_acc.append(acc)
    validation_f1.append(f1)
    print(f'\nAccuracy on validation set: {round(acc, 4)}\nF1 score on validation set: {round(f1, 4)}')
    print('Saving accuracy and f1 score on validation set\n' + '*' * 60)
    with open(f'track/' + str(try_round) + '/validation_acc.pkl', 'wb') as f:
        pickle.dump(validation_acc, f)
    with open(f'track/' + str(try_round) + '/validation_f1.pkl', 'wb') as f:
        pickle.dump(validation_f1, f)
    training_set, validation_set = prepare_dataset(validation=1)
#     print('\nShuffle training set')
#     training_set.shuffle(batch_size)  # shuffle training set

In [ ]:
outputs.argmax(1)

In [ ]:
colors_ids

In [ ]:
color_classifier.feature_fusion[0].weight.grad

In [ ]:
m = model.vision_tower.vision_model.encoder.layers[26].self_attn.v_proj.parametrizations.weight[0].lora_B
m.requires_grad

In [ ]:
model.vision_tower

In [ ]:
m - torch.tensor(model.vision_tower.vision_model.encoder.layers[25].self_attn.k_proj.parametrizations.weight[0].lora_A)

In [ ]:
validation_acc = [0.6005, 0.6156, 0.6094, 0.6152, 0.6028]

In [ ]:
with open(f'track/{try_round}/validation_acc.pkl', 'wb') as f:
    pickle.dump(validation_acc, f)

In [ ]:
correct_num_valid / total_num_valid

In [ ]:
print('Distracting features...')
# torch.autograd.set_detect_anomaly(True)
colors_list = []
cate_list = []
features_mat = []
epochs = 10
bs = 1
with torch.no_grad():
    # for epoch in range(epochs):
    training_set.shuffle(1)  # shuffle training set
    #     small_set = training_set.sample(list(range(3000)))
    #     small_set.shuffle(bs)
    #     tot = int(len(small_set))
    tot = int(len(training_set))
    for i, (image_paths, prompts, label_words, images) in enumerate(training_set):
        colors = training_set.baseColour[training_set.inds[i]]
        cates = training_set.subCategory[training_set.inds[i]]
        if colors[0] != 'Green' and colors[0] != 'Grey':
            continue
        torch.cuda.empty_cache()
        images = [images[:, :, :, i] for i in range(images.shape[3])]
        model_inputs = processor(prompts, images)
        model_inputs = move_inputs_to_device(model_inputs, device)
        input_ids = model_inputs['input_ids'].to(device)
        pixel_values = model_inputs['pixel_values'].half().to(device)

        features = model.distract_image_features(
            input_ids=input_ids,
            pixel_values=pixel_values)
        for j in range(bs):
            cate_list.append(cates[j])
            colors_list.append(colors[j])
            features_mat.append(torch.ravel(features[j]).cpu().numpy())
        print(f'{i}/{tot}', end='\r')
features_mat = np.array(features_mat)

In [ ]:
pca = PCA(n_components=2)
pca.fit(features_mat)
features_PCA = pca.transform(features_mat)
pca.explained_variance_ratio_.sum()

In [ ]:
color_drew = {}
for color in set(colors_list):
    color_drew[color] = np.random.random((1, 3))

In [ ]:
np.save('data/fashion-dataset/features_PCA.npy', features_PCA)
with open(f'data/fashion-dataset/colors_list.pkl', 'wb') as f:
    pickle.dump(colors_list, f)

In [ ]:
color_drew

In [ ]:
plt.figure()
# ax = plt.axes(projection="3d")
for i in range(features_PCA.shape[0]):
    point = features_PCA[i]
    color = colors_list[i]
    #     ax.scatter3D(point[0], point[1], point[2], color='b' if color=='Blue' else 'k')
    plt.scatter(point[0], point[1], c='g' if color == 'Green' else 'grey', s=2)
plt.show()

In [ ]:
from sklearn import svm

size = len(colors_list)
t_size = int(size * 0.8)
clf = svm.SVC(C=1, kernel='rbf', gamma=0.1)
clf.fit(features_PCA[:t_size], colors_list[:t_size])
pred = clf.predict(features_PCA[t_size:])
(np.array(pred) == np.array(colors_list[t_size:])).sum() / (size - t_size)